In [2]:
import sys
import os

# Assuming you know the installation path or can derive it
# Replace with your actual SDK installation path if different
sdk_install_path = r"C:\Program Files\HOLOEYE Photonics\SLM Display SDK (Python) v4.1.0" # Example path
python_api_path = os.path.join(sdk_install_path, "api", "python")

if python_api_path not in sys.path:
    sys.path.append(python_api_path)
    print(f"Added {python_api_path} to Python path.")

# Now you should be able to import hedslib
from hedslib.heds_types import *
import HEDS # If HEDS.py or the HEDS package is also directly in api/python

In [ ]:
import HEDS
from hedslib.heds_types import *
import sys
import os
import numpy as np
import random
import math
import time
import beam_comp
import serial
from beam_comp import *

# Assuming you know the installation path or can derive it
# Replace with your actual SDK installation path if different
sdk_install_path = r"C:\Program Files\HOLOEYE Photonics\SLM Display SDK (Python) v4.1.0" # Verify this!
python_api_path = os.path.join(sdk_install_path, "api", "python")

if python_api_path not in sys.path:
    sys.path.append(python_api_path)
    print(f"Added {python_api_path} to Python path.")

# 1. Initialize the SDK
err = HEDS.SDK.Init(4,1) # Using SDK version 4,1
if err != HEDSERR_NoError:
    print(f"Error initializing SDK: {HEDS.SDK.ErrorString(err)}")
    sys.exit() # Use sys.exit() to stop execution
else:
    print("SDK initialized successfully.")

slm = None # Initialize slm to None
try:
    # 2. Initialize an SLM
    # This call will typically open the SLM Device Detection GUI.
    # For automated use, consider using a preselection string like:
    # slm = HEDS.SLM.Init("-slm index:0 -nogui")
    slm = HEDS.SLM.Init()

    # Check if SLM initialization was successful
    if slm.errorCode() != HEDSERR_NoError:
        print(f"Error initializing SLM: {HEDS.SDK.ErrorString(slm.errorCode())}")
        sys.exit()
    else:
        # Access properties directly from the SLM object
        slm_width = slm.width_px() #
        slm_height = slm.height_px() #

        # To get the device name, you typically use the monitor info functions from SDK.libapi
        # Get the SLM window ID from the SLM object
        slm_window_id = slm.window().id() # The SLM object holds a reference to its SLMWindow

        # Get the monitor ID associated with this SLM window
        # The documentation for SDK.PrintMonitorInfos uses heds_info_monitor_get_id_used_slm
        monitor_id_slm = HEDS.SDK.libapi.heds_info_monitor_get_id_used_slm(slm_window_id)

        # Get the name of that monitor/device
        slm_device_name = HEDS.SDK.libapi.heds_info_monitor_get_name(monitor_id_slm)


        print(f"SLM connected successfully: {slm_device_name}")
        print(f"Resolution: {slm_width} x {slm_height}")

        # You can now display images, data, etc.
        # Example: show a blank screen (replace 128 with your desired gray value)
        # slm.showBlankScreen(128)
        # import time
        # time.sleep(5) # Keep it visible for 5 seconds

finally:
    # 3. Close the SLM window when done
    # if slm is not None:
    #     err = slm.window().close() #
    #     if err != HEDSERR_NoError:
    #         print(f"Error closing SLM window: {HEDS.SDK.ErrorString(err)}")
    #     else:
    #         print("SLM window closed.")
    if slm is not None:
        print("SLM Connected! You can now use it for your application.")
    # It's good practice to also close the SDK


SDK initialized successfully.
SLM connected successfully: HOLOEYE PLUTO-2.1 SLM
Resolution: 1920 x 1080
No SLM object to close (initialization might have failed).


In [ ]:
def display_phase_mask(slm_device, phase_mask_2d):
    """
    Displays a 2D NumPy array of phase values on an initialized SLM.

    Args:
        slm_device (SLM): An already initialized HOLOEYE SLM object.
        phase_mask_2d (numpy.ndarray): A 2D NumPy array where each value
                                       represents a phase in radians.
                                       The array should have a dtype of
                                       np.float32 for best performance.
    
    Returns:
        bool: True if the display was successful, False otherwise.
    """
    # Check if the SLM object is valid
    # if not isinstance(slm_device, SLM) or slm_device.errorCode() != HEDSERR_NoError:
    #     print("Error: The provided SLM device is not valid or has an error.")
    #     return False

    # Check if the input is a NumPy array
    if not isinstance(phase_mask_2d, np.ndarray):
        print("Error: The provided phase mask must be a NumPy array.")
        return False

    #print("Sending the 2D phase array to the SLM...")
    
    # The core function call to display the data
    error = slm_device.showPhaseData(phase_mask_2d.astype(np.float32))

    if error == HEDSERR_NoError:
        #print("Phase mask displayed successfully!")
        return True
    else:
        print(f"An error occurred while displaying the phase mask: {error}")
        return False

ser.close() # Close the serial port if it was opened previously
ser = serial.Serial('COM8', 115200, timeout=0.1)
time.sleep(0.05)  # Wait for Tiva to boot/reset


def photodiode_measurement(ser = ser, duration=0.05):

    adc_values = []
    start_time = time.time()

    while time.time() - start_time < duration:
        line = ser.readline().decode().strip()
        if line.isdigit():
            adc_values.append(int(line))



    if adc_values:
        return sum(adc_values) / len(adc_values)
    else:
        return None  # or 0 or raise an exception



def evaluate_objective_function(spin_config, interactions, slm = slm):
    """
    This function should be implemented to evaluate the objective function
    based on the measured energy from the photodiode or detector.
    
    Returns:
        float: The measured energy value.
    """
    mattis_phase_masks = interactions.generate_phase_masks(spin_config)
    mattis_energies = []
    for i in range(len(mattis_phase_masks)):
        display = display_phase_mask(slm, mattis_phase_masks[i].astype(np.float32))
        if not display:
            print("Error displaying phase mask on SLM. Returning high energy value.")
            return float('inf')
        time.sleep(0.05)  # Allow time for the SLM to update
        mattis_energies.append(photodiode_measurement()*interactions.eigvals[i])
    # Measure the energy from the photodiode or detector

    energy = sum(mattis_energies) if mattis_energies else 0
    print(f"energy measured: {energy}")
    return energy

def run_photonic_annealing(num_spins, evaluate_spin_vector_func, interaction, initial_temp, final_temp, cooling_rate, steps_per_temp):
    """
    Performs a simulated annealing algorithm using a hardware objective function.
    This function is agnostic to the hardware details (SLM, camera, etc.).

    Args:
        num_spins (int): The total number of spins in the system.
        evaluate_spin_vector_func (function): A function that takes a 1D spin vector,
                                              sends it to the hardware, and returns a
                                              measured energy (float).
        initial_temp (float): The starting temperature.
        final_temp (float): The ending temperature.
        cooling_rate (float): The factor by which temperature is multiplied.
        steps_per_temp (int): The number of Monte Carlo steps at each temperature.

    Returns:
        numpy.ndarray: The final, low-energy 1D spin vector.
    """
    # Initialize a 1D spin vector with random spins (-1 or 1)
    current_spin_vector = np.random.choice([-1, 1], size=num_spins)

    start_time = time.time()

    # Get the initial energy measurement by evaluating the initial spin vector
    print("Evaluating initial random spin configuration...")
    current_energy = evaluate_spin_vector_func(current_spin_vector)
    print(f"Initial measured energy: {current_energy:.4f}")

    temp = initial_temp
    energy_plot = []
    
    print("\nStarting photonic simulated annealing...")
    print(f"Initial Temp: {initial_temp}, Final Temp: {final_temp}, Cooling Rate: {cooling_rate}")

    while temp > final_temp:
        print(f"\nCurrent Temperature: {temp:.4f}")
        for step in range(steps_per_temp):
            # 1. Pick a random spin to consider flipping from the 1D vector
            idx = random.randint(0, num_spins - 1)
            
            # 2. Create a proposed new 1D vector with the flipped spin
            proposed_spin_vector = np.copy(current_spin_vector)
            proposed_spin_vector[idx] *= -1

            # 3. Evaluate the proposed state using the hardware function
            proposed_energy = evaluate_spin_vector_func(proposed_spin_vector, interaction)
            
            # 4. Calculate the change in energy
            delta_energy = proposed_energy - current_energy
            
            # 5. Decide whether to accept the flip using the Metropolis criterion
            if delta_energy < 0 or random.random() < math.exp(-delta_energy / temp):
                # Accept the new state
                current_spin_vector = proposed_spin_vector
                current_energy = proposed_energy

            energy_plot.append(current_energy)

            if step % 100 == 0:
                print(f"  Step {step}/{steps_per_temp} | Current Energy: {current_energy:.4f}")

        # Cool the system down
        temp *= cooling_rate
       
    print("\nSimulated annealing finished.")
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Total time taken: {elapsed_time:.2f} seconds")
    return current_spin_vector, energy_plot

In [9]:
filename = 'MC100.npz'

def data_loader(filename):
    """
    Takes data from .npz and returns adjacency matrix.
    """
    import numpy as np
    import scipy.sparse as sp

    # Load the .npz file
    data = np.load(filename, allow_pickle=True)

    # Reconstruct the sparse adjacency matrix (CSR format)
    adj_sparse = sp.csr_matrix(
        (data['data'], data['indices'], data['indptr']),
        shape=tuple(data['shape'])
    )

    # If you want a dense NumPy array (e.g., for neural nets)
    return adj_sparse.toarray()

Adj = data_loader(filename)



In [10]:
interactions = CompensatedMattisInteractions(Adj, 347,347)

In [14]:
interactions.prep()

interactions.generate_phase_masks([1 for i in range(100)])[1].astype(np.float32).shape


Preparing model...
  - SLM Layout: 10 rows x 10 cols
  - Macropixel Size: 192 x 108 pixels
Preparation complete.


MemoryError: Unable to allocate 7.91 MiB for an array with shape (1080, 1920) and data type float32

In [ ]:


final, energyvals = run_photonic_annealing(num_spins=100, evaluate_spin_vector_func=evaluate_objective_function, interaction= interactions, initial_temp=1000, final_temp=1, cooling_rate=0.995, steps_per_temp=10)

ser.close()

Evaluating initial random spin configuration...
energy measured: -14283.372045880504
Initial measured energy: -14283.3720

Starting photonic simulated annealing...
Initial Temp: 1000, Final Temp: 1, Cooling Rate: 0.995

Current Temperature: 1000.0000
energy measured: 5133.652210466607
  Step 0/10 | Current Energy: -14283.3720
energy measured: -87538.2616845106
energy measured: 7109.100524330162
energy measured: 15790.272483301407
energy measured: -28570.185553171425
energy measured: 33696.27389693173
energy measured: 11322.839625119843
energy measured: -21570.440000852366
energy measured: -12772.62781466222
energy measured: 19457.590636195586

Current Temperature: 1000.0000
energy measured: -30434.662166287657
  Step 0/10 | Current Energy: -87538.2617
energy measured: -24061.80315854249
energy measured: -62402.32567527033
energy measured: -40510.5451733418
energy measured: 14346.164332550892
energy measured: -23181.67701413705
energy measured: 63029.6636831761
energy measured: -81520.4

KeyboardInterrupt: 